In this notebook, we will discuss streaming large datasets into our Python environment. Traditionally, if you wanted to work with raster or shapefiles, you needed to download them beforehand and then import them directly from your local disk. There are two issues with this traditional approach. The first issue is the time spent downloading the file, and the second is the size of the file, which may end up being enormous. You may not want to work with the entire shapefile or raster file. Instead, you might only want to focus on a specific bounding box within that file. So, why spend time and memory on parts of the data that you don't need? This is why we would like to discuss streaming data directly into your script.

Now, let's focus on HRDEM collections in Datacube. These collections are in very fine resolutions, which means they contain valuable information for your projects and can improve the accuracy of geomatics calculations. However, they are huge! You need to spend a lot of time downloading them, and when you import them into your Python script, right off the bat, you may run into memory issues. How can we efficiently solve this issue?

The first step is to define the address of the Datacube STAC API endpoint. Then, we need to establish a connection to the API endpoint in order to access the collections within that endpoint. We use pystac_client, which is a Python package for working with STAC Catalogs and APIs that conform to the STAC and STAC API specs. PySTAC Client builds upon PySTAC through higher-level functionality and ability to leverage STAC API search endpoints.

In [19]:
import pystac_client
DC_URL = 'https://datacube.services.geo.ca/api'
# We establish a connection
catalog = pystac_client.Client.open(DC_URL)  

Now, with the catalog object, we are able to search the endpoint for any collection, bbox, date, etc., to find the data that we want to stream. You can think of this as manually using the STAC browser. 

Now we need to define our filters. How should all the data in the STAC API be filtered so that we only get the one(s) we want? First, we start with the data collection or collections that we want to search.

In [32]:
collections = ['hrdem-lidar']

Then, we can define a filter for temporal search and a filter for spatial search.

For the temporal search, we define the start date and an end date (inclusive) between which we want to retrieve data.

In [33]:
datetime_range = '2019-01-01/2024-12-31'

For the spatial search, we can either define a bbox or an AOI polygon for the area of interest. They should be in **lat/lon**.

In [34]:
bbox = [-114.00, 51.24, -113.94, 51.33]

In [35]:
aoi_polygon = {
    "type": "Polygon",
    "coordinates": [
        [
            [bbox[0], bbox[1]],  
            [bbox[2], bbox[1]],  
            [bbox[2], bbox[3]],
            [bbox[0], bbox[3]],
            [bbox[0], bbox[1]]   
        ]
    ]
}

Now, with these three basic filters, we can search the STAC API endpoint to retrieve the STAC items that may be available.

In [37]:
stac_items     = catalog.search(collections = collections, 
                                bbox = bbox, 
                                datetime = datetime_range
                               )

One STAC item has been found, and now it's time to read or stream the data into our script.

To stream the data into our script, we have three options:

-  Using the **odc-stac** library, which reads data and indirectly uses **GDAL** for handling geospatial data formats like GeoTIFFs or other raster-based formats.
-  Using the **stackstac.stack** library, which reads data and indirectly uses **GDAL** for handling geospatial data formats like GeoTIFFs or other raster-based formats.
-  Using the **rioxarray** library, which reads data and indirectly uses **rasterio** for handling geospatial data formats like GeoTIFFs or other raster-based formats.

But before diving into their differences and the requirements that should be met, what is the common feature of all these libraries? The common feature is the fact that these libraries read data directly into **Xarrays**, not NumPy arrays.

Xarray is built on top of Numpy and Pandas and adds extra capabilities for working with labeled, multi-dimensional arrays (such as climate data, satellite images, etc.).
It extends Numpy arrays by allowing metadata (coordinates, labels, etc.) to be attached to arrays. This makes it easier to work with data where dimensions have meaningful names, such as time, latitude, longitude, and variables.
In addition, Xarrays can be backed by Dask arrays, which support lazy evaluation. This means you can work with large datasets that don't fit in memory, performing computations while deferring the actual data reading until the end of the computation. Dask optimizes the workflow by parallelizing tasks, allowing efficient memory usage. It reads data in chunks rather than loading the entire dataset into memory at once, helping to improve both performance and memory efficiency.

- odc-stac:
  This library is primarily built for integrating STAC (SpatioTemporal Asset Catalog) items with the Open Data Cube (ODC) ecosystem. It is part of the ODC framework, which is used for managing and analyzing large-scale geospatial data.

- stackstac:
   stackstac is focused on efficiently stacking geospatial data from STAC (like satellite images or other temporal data) into a time-series or multi-temporal stack. It is lightweight compared to odc-stac and more specialized for handling STAC items and data for large-scale, multi-temporal analysis.

-  rioxarray:
  rioxarray is a general-purpose library for working with raster data in the xarray format. It integrates rasterio for reading and writing geospatial formats (like GeoTIFF) and leverages xarray for handling multi-dimensional arrays of data, which is useful for operations on raster datasets that involve multiple bands or time series.


Before diving into each of them, there is a requirement that should be met when we read HRDEM collections from the DataCube STAC API using **odc-stac** and **stackstac**, which are based on GDAL. GetGeoTransform and rasterio use different formats for reporting transform information. Order expected in proj:transform is the same as reported by rasterio. When using GDAL method you need to re-order in the following way:

In [38]:
def reorder_transform(gdal_transform):
    """
    Reorders the GDAL GeoTransform (6-element tuple) into the 9-element format
    that is compatible with proj:transform.
    """
    return [gdal_transform[1], gdal_transform[2], gdal_transform[0],  # x scale (a), rotation about y (b), x-coordinate of upper left (c)
            gdal_transform[4], gdal_transform[5], gdal_transform[3],  # rotation about x (d), y scale (e), y-coordinate of upper left (f)
            0, 0, 1]  # The last part is fixed for affine transformation matrix (0, 0, 1)

In [45]:
stac_found_items = [item for item in stac_items.items()]
for item in stac_found_items:
    if "proj:transform" in item.properties:
        proj_transform = item.properties["proj:transform"]
        if isinstance(proj_transform, list):
            
            reordered_transform = reorder_transform(proj_transform)
            item.properties["proj:transform"] = reordered_transform
            print(f"Updated proj:transform for item {item.id}")
        
       
        elif isinstance(proj_transform, rasterio.Affine):
            
            proj_transform_reordered = reorder_affine(proj_transform)  
            item.properties["proj:transform"] = proj_transform_reordered
            print(f"Updated proj:transform (Affine) for item {item.id}")
    else:
        print(f"Item {item.id} does not have proj:transform in properties.")

Updated proj:transform for item NRCAN-Calgary_East_utm12_2020-1m
Updated proj:transform for item NRCAN-Calgary_West_utm11_2020-1m


Now, we begin by reading the data using odc-stac. Beforehand, we may want to review the available bands for the item(s) and then select only the bands that we need.

In [46]:
for item in stac_found_items:
    print(item.assets.keys())

dict_keys(['dsm', 'dtm', 'extent', 'dsm-vrt', 'dtm-vrt', 'coverage', 'thumbnail'])
dict_keys(['dsm', 'dtm', 'extent', 'dsm-vrt', 'dtm-vrt', 'coverage', 'thumbnail'])


In [47]:
import odc.stac

xrDS = odc.stac.load(stac_found_items,
                     chunks     = {"x": 100, "y": 1000},
                     bands      = ["dsm", "dtm"],
                     bbox       = bbox,
                     resolution = 1.0)

In [48]:
xrDS

<xarray.Dataset> Size: 590MB
Dimensions:      (y: 10713, x: 6887, time: 1)
Coordinates:
  * y            (y) float64 86kB 4.527e+05 4.527e+05 ... 4.42e+05 4.42e+05
  * x            (x) float64 55kB -1.297e+06 -1.297e+06 ... -1.29e+06 -1.29e+06
    spatial_ref  int32 4B 3979
  * time         (time) datetime64[ns] 8B 2020-10-08T12:00:00
Data variables:
    dsm          (time, y, x) float32 295MB dask.array<chunksize=(1, 1000, 100), meta=np.ndarray>
    dtm          (time, y, x) float32 295MB dask.array<chunksize=(1, 1000, 100), meta=np.ndarray>

So, we streamed our data using odc.stac.load. There are a couple of arguments that helped us streamline the data stream according to our filters and preferences.

1. **bands**: As you can see, each STAC item has 9 bands: ['dsm', 'dtm', 'extent', 'dsm-vrt', 'dtm-vrt', 'coverage', 'thumbnail', 'hillshade-dsm', 'hillshade-dtm']. Importing unnecessary bands may cause memory and speed issues. We can benefit from the bands argument and only import the bands that we need, disregarding the rest.
2. **bbox**: Instead of importing the large HRDEM dataset, we can import only the region of interest. This helps us manage the speed and memory usage of our program.
3. **resolution** : This argument should be defined whenever we have the bbox argument set.
4. **chunks** : This is an interesting argument, and you need to set this whenever you want your xarray to be backed by a dask array. This means whenever you want to enable lazy loading and benefit from dask's parallel computing under the hood. The chunk size is essentially the definition for dask on how to read data. When we set 1000 for both x and y, dask reads a box of 100 by 100 of the data, instead of reading the entire array. And with parallel computing, it reads multiple chunks simultaneously. So, you're increasing speed and controlling memory usage.

When you look at xrDS, you do not see the data in the dask arrays. At this point, the data has not been read **yet**. We've defined a lens to look at the data. The metadata, the array shape, and everything are set, except the data itself. You can run calculations on these datasets and get new xarrays (dask arrays), but the data is not yet loaded. When do we get the data?

Imagine you have xrDS, and then you run Calculations A, B, and C to get xrDS_C. Now you actually want to see the data. What you need to do is run **.compute()** on xrDS_C, and as soon as you do that, data starts flowing into the workflow you've created. Each of those intermediate arrays gets their data and modifies the data until you get xrDS_C.

This is cool because when you create a dask workflow and then run .compute() on the last array, dask optimizes everything for you. It evaluates the workflow and runs it in the most efficient way computationally. So, there is a lot of optimization regarding resource usage!

In [49]:
xrDS.compute()

<xarray.Dataset> Size: 590MB
Dimensions:      (y: 10713, x: 6887, time: 1)
Coordinates:
  * y            (y) float64 86kB 4.527e+05 4.527e+05 ... 4.42e+05 4.42e+05
  * x            (x) float64 55kB -1.297e+06 -1.297e+06 ... -1.29e+06 -1.29e+06
    spatial_ref  int32 4B 3979
  * time         (time) datetime64[ns] 8B 2020-10-08T12:00:00
Data variables:
    dsm          (time, y, x) float32 295MB 1.089e+03 1.089e+03 ... 1.113e+03
    dtm          (time, y, x) float32 295MB 1.089e+03 1.089e+03 ... 1.113e+03

Now the data is there!

We continue by reading the data using stackstac. 

In [ ]:
import stackstac
xrDS_ss = stackstac.stack(stac_found_items, 
                          assets        = ["dsm", "dtm"], 
                          resolution    = 1.0,
                          bounds_latlon = bbox, 
                          chunksize     = (1000, 1000))

In [51]:
xrDS_ss

<xarray.DataArray 'stackstac-2b17ed0c7ba5282c04b5b6768974f34c' (time: 2,
                                                                band: 2,
                                                                y: 10713,
                                                                x: 6887)> Size: 2GB
dask.array<fetch_raster_window, shape=(2, 2, 10713, 6887), dtype=float64, chunksize=(1, 1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * time           (time) datetime64[ns] 16B 2020-10-08T12:00:00 2020-10-08T1...
    id             (time) <U32 256B 'NRCAN-Calgary_East_utm12_2020-1m' 'NRCAN...
  * band           (band) <U3 24B 'dsm' 'dtm'
  * x              (x) float64 55kB -1.297e+06 -1.297e+06 ... -1.29e+06
  * y              (y) float64 86kB 4.527e+05 4.527e+05 ... 4.42e+05 4.42e+05
    proj:geometry  (time) object 16B {'type': 'Polygon', 'coordinates': [[[-1...
    ...             ...
    proj:epsg      int64 8B 3979
    created        (time) <U20 160B '2024-01-16T14:28:33Z' '2024-01-16T14:29:...
    collection     <U11 44B 'hrdem-lidar'
    description    (band) <U131 1kB "Digital Surface Model derived from Airbo...
    title          (band) <U63 504B 'Digital Surface Model (COG) / Modèle num...
    epsg           int64 8B 3979
Attributes:
    spec:        RasterSpec(epsg=3979, bounds=(-1296659.0, 442022.0, -1289772...
    crs:         epsg:3979
    transform:   | 1.00, 0.00,-1296659.00|\n| 0.00,-1.00, 452735.00|\n| 0.00,...
    resolution:  1.0

The arguments seem pretty much alike. But with stackstac, you'll get many more attributes saved in your xarray.

In [52]:
xrDS_ss.compute()

<xarray.DataArray 'stackstac-2b17ed0c7ba5282c04b5b6768974f34c' (time: 2,
                                                                band: 2,
                                                                y: 10713,
                                                                x: 6887)> Size: 2GB
array([[[[          nan,           nan,           nan, ...,
          1126.0234375 , 1125.99694824, 1125.97814941],
         [          nan,           nan,           nan, ...,
          1126.02148438, 1125.98693848, 1125.96179199],
         [          nan,           nan,           nan, ...,
          1126.02001953, 1125.98083496, 1125.95092773],
         ...,
         [1084.30627441, 1084.41137695, 1084.48864746, ...,
          1112.70629883, 1112.70898438, 1112.71044922],
         [1084.27502441, 1084.37817383, 1084.4543457 , ...,
          1112.71801758, 1112.71923828, 1112.72363281],
         [1084.25231934, 1084.3482666 , 1084.42797852, ...,
          1112.72314453, 1112.72583008, 1112.73278809]],

        [[          nan,           nan,           nan, ...,
          1125.99267578, 1125.96936035, 1125.9440918 ],
         [          nan,           nan,           nan, ...,
          1125.98730469, 1125.95874023, 1125.93127441],
         [          nan,           nan,           nan, ...,
          1125.98693848, 1125.9519043 , 1125.92871094],
...
         [1084.30603027, 1084.40991211, 1084.48583984, ...,
                    nan,           nan,           nan],
         [1084.27160645, 1084.3770752 , 1084.45153809, ...,
                    nan,           nan,           nan],
         [1084.24938965, 1084.34667969, 1084.42480469, ...,
                    nan,           nan,           nan]],

        [[1088.91674805, 1088.90930176, 1088.91320801, ...,
                    nan,           nan,           nan],
         [1088.9005127 , 1088.90808105, 1088.9095459 , ...,
                    nan,           nan,           nan],
         [1088.89892578, 1088.90197754, 1088.91235352, ...,
                    nan,           nan,           nan],
         ...,
         [1084.26306152, 1084.3684082 , 1084.45251465, ...,
                    nan,           nan,           nan],
         [1084.22839355, 1084.33789062, 1084.42333984, ...,
                    nan,           nan,           nan],
         [1084.20703125, 1084.31567383, 1084.3918457 , ...,
                    nan,           nan,           nan]]]])
Coordinates: (12/13)
  * time           (time) datetime64[ns] 16B 2020-10-08T12:00:00 2020-10-08T1...
    id             (time) <U32 256B 'NRCAN-Calgary_East_utm12_2020-1m' 'NRCAN...
  * band           (band) <U3 24B 'dsm' 'dtm'
  * x              (x) float64 55kB -1.297e+06 -1.297e+06 ... -1.29e+06
  * y              (y) float64 86kB 4.527e+05 4.527e+05 ... 4.42e+05 4.42e+05
    proj:geometry  (time) object 16B {'type': 'Polygon', 'coordinates': [[[-1...
    ...             ...
    proj:epsg      int64 8B 3979
    created        (time) <U20 160B '2024-01-16T14:28:33Z' '2024-01-16T14:29:...
    collection     <U11 44B 'hrdem-lidar'
    description    (band) <U131 1kB "Digital Surface Model derived from Airbo...
    title          (band) <U63 504B 'Digital Surface Model (COG) / Modèle num...
    epsg           int64 8B 3979
Attributes:
    spec:        RasterSpec(epsg=3979, bounds=(-1296659.0, 442022.0, -1289772...
    crs:         epsg:3979
    transform:   | 1.00, 0.00,-1296659.00|\n| 0.00,-1.00, 452735.00|\n| 0.00,...
    resolution:  1.0

The last option is streaming data using rioxarray. This library is based on rasterio, and it can read data in xarray format as well. For streaming the data using rioxarray, we do not have as many options for defining the bbox or resolution. What we work with is the href of our STAC items, and after streaming the whole dataset, we can clip it to the bounding box of interest.

In addition, we cannot stack the items. We need to read each item into a separate xarray dataset. So, we start with the first STAC item in our STAC items list.

In [60]:
dtm_url = None
for asset_key, asset in stac_items[0].assets.items():
    print(f"URL for {asset_key} is {asset.href}")
    if asset_key == "dtm":
        dtm_url = asset.href

URL for dsm is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-dsm.tif
URL for dtm is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-dtm.tif
URL for extent is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-extent.geojson
URL for dsm-vrt is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-dsm.vrt
URL for dtm-vrt is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-dtm.vrt
URL for coverage is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_utm12_2020-1m-coverage.gpkg
URL for thumbnail is https://datacube-prod-data-public.s3.amazonaws.com/store/elevation/hrdem/hrdem-lidar/NRCAN-Calgary_East_

In [65]:
import rioxarray
bbox_pr = [-1296660.0, 443357.0, -1289951.0, 452669.35559263]
xds_rio = rioxarray.open_rasterio(
        dtm_url,
        chunks={'x': 1000, 'y': 1000},
).rio.clip_box(*bbox_pr)

xds_rio

<xarray.DataArray (band: 1, y: 9313, x: 6709)> Size: 250MB
dask.array<getitem, shape=(1, 9313, 6709), dtype=float32, chunksize=(1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 54kB -1.297e+06 -1.297e+06 ... -1.29e+06 -1.29e+06
  * y            (y) float64 75kB 4.527e+05 4.527e+05 ... 4.434e+05 4.434e+05
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_DATETIME:  2020:10:08 12:00:00
    AREA_OR_POINT:     Area
    scale_factor:      1.0
    add_offset:        0.0
    _FillValue:        -32767.0

Along with the mentioned points for when we work with rioxarray, another important point is that the bbox used for clipping data should be in the projected coordinate system, not in lat/lon coordinates.

So we have introduced three different ways and libraries of streaming Datacube collection datasets into your Python scripts instead of downloading them beforehand. In the next notebook, we will show some examples of basic classifications on the Dask array that we just created.